# Scrape survey information from yougov website
* Yougov has a public API - in some cases this gives results via a json, in other cases you just get a link to a pdf document.

In [7]:
import requests
from tabula import read_pdf
from tika import parser
import re
import pycountry
import json


### 0. testing the api
Yougov API call. Data is available via an API so call be pulled in easily.

In [2]:
basic_url = "https://yougov.co.uk/_pubapis/v5/uk/search/content/surveys/?category=overview&kind=editorial&offset=0&limit=20"
x = requests.get(basic_url)
basic_response = x.json()
print(basic_response.keys())
print(basic_response['hits'])

dict_keys(['data', 'hits', 'totals'])
13697


### 1. Extracting all data

In [9]:
def scrape_yougov(url):
    x = requests.get(url)
    response = x.json()
    return response
    

In [10]:
basic_url = "https://yougov.co.uk/_pubapis/v5/uk/search/content/surveys/?category=overview&kind=editorial&offset=0&limit=20"
x = requests.get(basic_url)
basic_response = x.json()

for x in range(0,10000,20):
    url = "https://yougov.co.uk/_pubapis/v5/uk/search/content/surveys/?category=overview&kind=editorial&offset="+str(x)+"&limit=20"
    response = scrape_yougov(url)
    basic_response['data'].extend(response['data'])
    

Dump data into a json file

In [ ]:
with open('C:\\Users\\lundr\\survey_search\\data\\yougov_070822.json', 'w') as json_file:
    json.dump(basic_response['data'], json_file)

Response is collected into a list of dictionaries

In [6]:
basic_response['data'][0:20]

[{'_final_score': 1.0,
  '_id': 'dfecc2b2-14a8-11ed-8aa9-e35f3ffa2395',
  '_native_score': 1.0,
  'cms_categories': ['politics'],
  'doc_type': 'result',
  'primary_category': 'politics',
  'primary_related_entity': {'image': 'https://d3hjzzsa8cr26l.cloudfront.net/55df9724-0226-11e9-b0b1-edf898abdd42.jpg?zcw=363&zch=363&zct=0&zcl=95',
   'name': 'Economy',
   'primary_type': {'linkable': True,
    'name': 'Issue',
    'reviewable': True,
    'uuid': 'bb13b601-a904-11e1-9412-005056900141'},
   'url': 'Economy-Issue',
   'uuid': 'bb744c9f-a904-11e1-9412-005056900141',
   'virtual_type_uuid': '04b59722-9f5b-11e8-b8e9-0fd00287d121'},
  'published_at': '2022-08-05T14:30:00Z',
  'question_id': 3,
  'related_entities': [{'image': 'https://d3hjzzsa8cr26l.cloudfront.net/55df9724-0226-11e9-b0b1-edf898abdd42.jpg?zcw=363&zch=363&zct=0&zcl=95',
    'name': 'Economy',
    'primary_type': {'linkable': True,
     'name': 'Issue',
     'reviewable': True,
     'uuid': 'bb13b601-a904-11e1-9412-005056900

the 'title' key tells you the question
the 'doc_type' key tells you whether it is a 'result'(all results provided in json) or a 'document' (a link to a pdf is supplied).

In [19]:
basic_response['data'][0]['title']

'Imagining that the UK entered a recession in the next 12 months, who do you think would be best able to end such a recession?'

In [16]:
basic_response['data'][0]['doc_type']

'result'

In [20]:
other = 0
docs = 0
result = 0
for el in basic_response['data']:
    if el['doc_type'] == 'document':
        docs +=1
    elif el['doc_type'] == 'result':
        result +=1
    else:
        other +=1

In [21]:
print(docs)
print(result)
print(other)

5464
4536
0


In [16]:
t = read_pdf('https://docs.cdn.yougov.com/icsj857kyv/YouGov%20-%20Misogyny%20Hate%20Crime%20Survey%20Results.pdf',stream=True)
type(t)

'pages' argument isn't specified.Will extract only from page 1 by default.


list

In [89]:
 # pip install tika

raw = parser.from_file('https://docs.cdn.yougov.com/icsj857kyv/YouGov%20-%20Misogyny%20Hate%20Crime%20Survey%20Results.pdf')
print(raw['content'])

2022-08-06 22:58:20,582 [MainThread  ] [INFO ]  Retrieving https://docs.cdn.yougov.com/icsj857kyv/YouGov%20-%20Misogyny%20Hate%20Crime%20Survey%20Results.pdf to C:\Users\lundr\AppData\Local\Temp/icsj857kyv-yougov-20-20misogyny-20hate-20crime-20survey-20results.pdf.












































Internal_PoliceBill_220209.xls


YouGov Survey Results

Sample Size: 1665 adults in GB
Fieldwork: 8th - 9th February 2022

Total Con Lab
Lib 

Dem
Remain Leave Male Female 18-24 25-49 50-64 65+ ABC1 C2DE London Rest of South

Midlands / 
Wales

North Scotland

Weighetd Sample 1665 564 413 150 618 659 809 856 181 694 401 388 949 716 200 559 361 401 143
Unweighted Sample 1665 552 426 152 654 665 737 928 161 691 396 417 972 693 196 561 375 388 145

% % % % % % % % % % % % % % % % % % %

Under current law, judges must consider a longer 
sentence if the victim of the crime was targeted 
due to their race, religion, sexual orientation, 
disability, or transgender identity. To what extent, if 
at all, would you support or oppose longer 
sentences being given if the crime was motivated 
by hostility or prejudice based on sex or gender? 

Strongly support 40 39 44 43 47 36 37 43 29 40 42 41 39 41 45 38 40 39 40
Somewhat support 32 32 34 35 31 31 31 32 3

In [90]:
string = raw['content']
pattern="\n\n"

In [91]:
split_string = string.split(pattern)

In [92]:

for x in split_string:
    if '?' in x:
        print(x)
    else:
        pass

Under current law, judges must consider a longer 
sentence if the victim of the crime was targeted 
due to their race, religion, sexual orientation, 
disability, or transgender identity. To what extent, if 
at all, would you support or oppose longer 
sentences being given if the crime was motivated 
by hostility or prejudice based on sex or gender? 


In [15]:
def extract_questions_from_yougov_pdf(pdf_file):
    questions= []
    raw = parser.from_file(pdf_file)
    string = raw['content']
    pattern="\n\n"
    split_string = string.split(pattern)
    for x in split_string:
        if '?' in x:
            x = x.replace('\n','')
            questions.append(x)
        else:
            pass
    return questions

In [16]:
pdf_file= 'https://docs.cdn.yougov.com/icsj857kyv/YouGov%20-%20Misogyny%20Hate%20Crime%20Survey%20Results.pdf'

test = extract_questions_from_yougov_pdf(pdf_file)

print(test)

2022-08-07 21:09:48,461 [MainThread  ] [INFO ]  Retrieving https://docs.cdn.yougov.com/icsj857kyv/YouGov%20-%20Misogyny%20Hate%20Crime%20Survey%20Results.pdf to C:\Users\lundr\AppData\Local\Temp/icsj857kyv-yougov-20-20misogyny-20hate-20crime-20survey-20results.pdf.


['Under current law, judges must consider a longer sentence if the victim of the crime was targeted due to their race, religion, sexual orientation, disability, or transgender identity. To what extent, if at all, would you support or oppose longer sentences being given if the crime was motivated by hostility or prejudice based on sex or gender? ']


In [18]:
test_response = basic_response['data'][0:100]

In [ ]:
for el in basic_response['data']:
    if el['doc_type'] == 'document':
        el['questions'] = extract_questions_from_yougov_pdf(el['url'])
    elif el['doc_type'] == 'result':
        el['questions'] =  el['title']

2022-08-07 21:23:36,751 [MainThread  ] [INFO ]  Retrieving https://docs.cdn.yougov.com/zqo0x3y6r7/TheTimesResults_220802_ConMembers_final_W2.pdf to C:\Users\lundr\AppData\Local\Temp/zqo0x3y6r7-thetimesresults_220802_conmembers_final_w2.pdf.
2022-08-07 21:23:37,287 [MainThread  ] [INFO ]  Retrieving https://docs.cdn.yougov.com/ugsrx0iild/YouGov%20Results%20-%20Tattoos.pdf to C:\Users\lundr\AppData\Local\Temp/ugsrx0iild-yougov-20results-20-20tattoos.pdf.
2022-08-07 21:23:37,552 [MainThread  ] [INFO ]  Retrieving https://docs.cdn.yougov.com/oagf5o7v1e/CAMRA_Pint_220722_fieldwork.pdf to C:\Users\lundr\AppData\Local\Temp/oagf5o7v1e-camra_pint_220722_fieldwork.pdf.
2022-08-07 21:23:37,704 [MainThread  ] [INFO ]  Retrieving https://docs.cdn.yougov.com/7m57n8mv55/TimesRadio_Favourability_20220727_Results.pdf to C:\Users\lundr\AppData\Local\Temp/7m57n8mv55-timesradio_favourability_20220727_results.pdf.
2022-08-07 21:23:37,899 [MainThread  ] [INFO ]  Retrieving https://docs.cdn.yougov.com/xav12v

In [ ]:
with open('C:\\Users\\lundr\\survey_search\\data\\yougov_070822_wqs.json', 'w') as json_file:
    json.dump(basic_response['data'], json_file)

In [103]:
countries = []
alpha_2 = []
alpha_3 = []
for e in list(pycountry.countries):
    countries.append(e.name)
    alpha_2.append(e.alpha_2)
    alpha_3.append(e.alpha_3)
    
for c in countries:
    if c in string:
        print(c)
# for d in alpha_2:
#     if d in string:
#         print(d)  
# for f in alpha_3:
#     if f in string:
#         print(f)         
        
    else:
        pass
        

AL
DE
GB
GE
SE
TO


In [94]:
list(pycountry.countries)

[Country(alpha_2='AW', alpha_3='ABW', flag='🇦🇼', name='Aruba', numeric='533'),
 Country(alpha_2='AF', alpha_3='AFG', flag='🇦🇫', name='Afghanistan', numeric='004', official_name='Islamic Republic of Afghanistan'),
 Country(alpha_2='AO', alpha_3='AGO', flag='🇦🇴', name='Angola', numeric='024', official_name='Republic of Angola'),
 Country(alpha_2='AI', alpha_3='AIA', flag='🇦🇮', name='Anguilla', numeric='660'),
 Country(alpha_2='AX', alpha_3='ALA', flag='🇦🇽', name='Åland Islands', numeric='248'),
 Country(alpha_2='AL', alpha_3='ALB', flag='🇦🇱', name='Albania', numeric='008', official_name='Republic of Albania'),
 Country(alpha_2='AD', alpha_3='AND', flag='🇦🇩', name='Andorra', numeric='020', official_name='Principality of Andorra'),
 Country(alpha_2='AE', alpha_3='ARE', flag='🇦🇪', name='United Arab Emirates', numeric='784'),
 Country(alpha_2='AR', alpha_3='ARG', flag='🇦🇷', name='Argentina', numeric='032', official_name='Argentine Republic'),
 Country(alpha_2='AM', alpha_3='ARM', flag='🇦🇲', 

In [24]:
print(raw['content'])











































Results for YouGov Cambridge - Globalism 2021 - OMGLOB131 - YouGov tables - All countries (1).xlsx


YouGov Cambridge
The Globalism Project - All countries
Fieldwork Dates: 4th August - 21st September 2021

France Germany Sweden Denmark Spain Italy Greece Hungary

Generally speaking, compared with before the Coronavirus 
pandemic, are you doing more or less of each of the 
following, or has there been no real change from before? 
(Please select one option on each row)

Sleeping

Unweighted base 1085 1009 1038 1170 1035 1023 1045 1012

Base: All country adults 1085 1009 1038 1170 1035 1023 1045 1012

Doing much more 6% 7% 3% 1% 5% 5% 7% 4%

Doing a little more 13% 16% 14% 11% 13% 12% 15% 11%

No change 67% 66% 70% 81% 54% 60% 58% 62%

Doing a little less 8% 8% 8% 4% 20% 15% 13% 12%

Doing much less 3% 2% 3% 2% 6% 6% 5% 7%

Don't know 2% 2% 1% 1% 1% 2% 1% 3%

Prefer not to say - - - - - - - -

Net: More 19% 23% 18% 12% 19% 17% 22% 15%

Net: Less 

In [9]:
  data_dict[k] = read_pdf(links_dict[k],pages=[first,second])

NameError: name 'read_pdf' is not defined

In [6]:
import sys
print(sys.prefix)

C:\Users\lundr\Miniconda3\envs\survey
